Randomly separates the file list into train and test sets, while maintaining the demographics of the full set of files.

In [1]:
#imports
import csv 
import random
import pandas as pd
import numpy as np

In [2]:
#get and shuffle the file
filename = '110524.csv'

df = pd.read_csv(filename)

# Shuffle the DataFrame rows randomly
df_shuffled = df.sample(frac=1, random_state=random.seed())


In [3]:
#get statistics
#counts = df.groupby(['language', 'emotion']).size()
counts = df.groupby(['language', 'emotion']).size().reset_index(name='count')

#define what percent of the data will be pulled out as test data
pcttest = .1

#define the distribution of files that we want in the final set
counts['dset'] = (counts['count']*pcttest).astype(int)
print(counts)

   language emotion  count  dset
0       ENG     ANG    399    39
1       ENG     HAP    400    40
2       ENG     SAD    400    40
3       EST     ANG    131    13
4       EST     HAP    134    13
5       EST     SAD    121    12
6       FRE     ANG     36     3
7       FRE     HAP     36     3
8       FRE     SAD     36     3
9       GER     ANG    127    12
10      GER     HAP     71     7
11      GER     SAD     62     6
12      GRE     ANG    121    12
13      GRE     HAP    119    11
14      GRE     SAD    122    12
15      ITA     ANG     73     7
16      ITA     HAP     72     7
17      ITA     SAD     73     7


In [4]:
counts['movelist'] = 0 #set up row to store how many got moved
testrows = [] #list of rows we want to remove

#Make a list of the first rows that meet the criteria above
for i in range(len(df_shuffled)):
    lang = df_shuffled['language'].iloc[i]
    emot = df_shuffled['emotion'].iloc[i]
    #check to see if we need more of that combo
    currentpull = counts.loc[(counts['language'] == lang) & (counts['emotion'] == emot), 'movelist'].item()
    needed = counts.loc[(counts['language'] == lang) & (counts['emotion'] == emot), 'dset'].item()
    if currentpull<needed:
        #put this row in the list for the test set
        testrows.append(i)
        #update the the counts df
        counts.loc[(counts['language'] == lang) & (counts['emotion'] == emot), 'movelist'] += 1
print(len(testrows))

247


In [5]:
#set up a data frame to receive the rows that we want moved
# Create a new DataFrame to store the moved rows
testset = df_shuffled.iloc[testrows]

# Remove the moved rows from the original DataFrame
trainset = df_shuffled.drop(testrows)

print(len(testset), len(trainset))
#check the test set distribution
testcounts = testset.groupby(['language', 'emotion']).size().reset_index(name='count')
print(testcounts)

247 2286
   language emotion  count
0       ENG     ANG     39
1       ENG     HAP     40
2       ENG     SAD     40
3       EST     ANG     13
4       EST     HAP     13
5       EST     SAD     12
6       FRE     ANG      3
7       FRE     HAP      3
8       FRE     SAD      3
9       GER     ANG     12
10      GER     HAP      7
11      GER     SAD      6
12      GRE     ANG     12
13      GRE     HAP     11
14      GRE     SAD     12
15      ITA     ANG      7
16      ITA     HAP      7
17      ITA     SAD      7


In [89]:
# Write the two separate sets back to a CSV file
trainset.to_csv('trainset.csv', index=False)
testset.to_csv('testset.csv', index=False)